## Parsing rare list

https://globalgenes.org/rarelist


In [1]:
# Fetch HTML using requests lib and feed to bs4
import requests
result = requests.get("https://globalgenes.org/rarelist")

from bs4 import BeautifulSoup
from bs4 import NavigableString
soup = BeautifulSoup(result.content, 'html.parser')



In [2]:
# check
soup.title

<title>Rare Disease List</title>

In [3]:
# write formatted html to file
# (not used: this is just a useful side effect for exploration)
f=open('rarelist.html','w')
f.write(soup.prettify())
f.close()

In [26]:
# use bs4 to extract names from HTML

names = []  ## all disease names found
name2url = {}  ## mapping of names to URLs

h5s = soup.find_all("h5")
for h5 in h5s:
    ul = h5.find_next_sibling('ul')
    for li in ul.findAll('li'):
        if len(li.contents) == 0:
            continue
        n = li.contents[0]
        if n is None:
            print('BAD: {}'.format(li))
            continue
        if not isinstance(n, NavigableString):
            n = n.contents[0]
            if li.select('a'):
                url = li.a['href']
            
                name2url[n] = url
        names.append(n)
        
# show the first 20 for sanity checking
names[0:20]

['Aagenaes syndrome',
 'Aarskog syndrome',
 'Aase Smith syndrome',
 'ABCD syndrome',
 'Abderhalden Kaufmann Lignac syndrome',
 'Abdominal aortic aneurysm',
 'Abdominal chemodectomas with cutaneous angiolipomas',
 'Abdominal cystic lymphangioma',
 'Abdominal obesity metabolic syndrome',
 'Aberrant subclavian artery',
 'Abetalipoproteinemia',
 'Abidi X-linked mental retardation syndrome',
 'Ablepharon macrostomia syndrome',
 "Abrikosov's tumor",
 'Abruzzo Erickson syndrome',
 'Absence of fingerprints congenital milia',
 'Absence of gluteal muscle',
 'Absence of septum pellucidum',
 'Absence of Tibia',
 'Absence of tibia with polydactyly']

In [5]:
## sanity check URL mapping
list(name2url.items())[0:10]

[('Acute disseminated encephalomyelitis', 'http://ulf.org/'),
 ('Acute hemorrhagic leukoencephalitis', 'http://ulf.org/'),
 ('Adrenoleukodystrophy X-linked', 'http://ulf.org/'),
 ('Adrenomyeloneuropathy', 'http://ulf.org/'),
 ('Aicardi-Goutieres syndrome', 'http://ulf.org/'),
 ('Alexander disease', 'http://ulf.org/'),
 ('Alkaptonuria', 'http://www.alkaptonuria.info/'),
 ('Alpers syndrome',
  'http://www.umdf.org/site/c.8qKOJ0MvF7LUG/b.7929671/k.BDF0/Home.htm'),
 ('Alzheimer disease familial', 'http://www.mitoaction.org/'),
 ('Alzheimer disease type 1', 'http://www.mitoaction.org/')]

In [31]:
import csv
with open('rare-list.tsv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='\t')
    for n in names:
        spamwriter.writerow([n, name2url.get(n)])

In [6]:
## use ontobio lib for fetching ontologies and lexical mapping
from ontobio import OntologyFactory

/usr/local/lib/python3.6/site-packages/cachier/mongo_core.py:24: UserWarning: Cachier warning: pymongo was not found. MongoDB cores will not work.
  "Cachier warning: pymongo was not found. MongoDB cores will not work.")


In [7]:
ofa = OntologyFactory()

In [8]:
hp = ofa.create('obo:hp')

In [9]:
mondo = ofa.create('obo:mondo')

In [10]:
from ontobio.lexmap import LexicalMapEngine
lexmap = LexicalMapEngine()


In [11]:
# Quick hack to make a degenerate 'ontology' from the list of names
from ontobio import Ontology

def ont_from_names(names):
    ont = Ontology(id='rare')
    for n in names:
        ## use name as ID
        ont.add_node(n, n)
    return ont
        
rare = ont_from_names(names)
rare

rare handle: None meta: None

In [12]:
## quick inspection
rare.nodes()[0:10]

['Aagenaes syndrome',
 'Aarskog syndrome',
 'Aase Smith syndrome',
 'ABCD syndrome',
 'Abderhalden Kaufmann Lignac syndrome',
 'Abdominal aortic aneurysm',
 'Abdominal chemodectomas with cutaneous angiolipomas',
 'Abdominal cystic lymphangioma',
 'Abdominal obesity metabolic syndrome',
 'Aberrant subclavian artery']

In [13]:
## index the 3 ontologies
lexmap.index_ontology(hp)
lexmap.index_ontology(mondo)
lexmap.index_ontology(rare)

ERROR:root:Use meaningful ids if label not present: HP:0002503 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0002829 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001649 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0009125 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0005390 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001789 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0003470 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://www.w3.org/2002/07/owl#Thing "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001417 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0000006 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0000007 "None" label

ERROR:root:Use meaningful ids if label not present: MFOMD:0000122 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0011031 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0002090 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: NCIT:C28079 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001251 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001254 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0007313 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001289 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0003401 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/OMIM_113970 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001397 "

ERROR:root:Use meaningful ids if label not present: HP:0000252 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: UBERON:0016551 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: UBERON:0005854 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: UBERON:0005853 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0000278 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: UBERON:0005852 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0000975 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0002617 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0000989 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001894 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001635 "None" label None Non

ERROR:root:Use meaningful ids if label not present: HP:0002373 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0011220 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/ECTO_0000007 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0010465 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/ECTO_0000006 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0010864 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0008035 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0002242 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0100806 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: NBO:0000433 "None" label None None 1.0
ERROR:root:Use meaningful ids if 

ERROR:root:Use meaningful ids if label not present: HP:0007858 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0008734 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001627 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0006685 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001967 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0000526 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001644 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0001297 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0003355 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0000768 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: HP:0002205 "None" label None None 1.0
ERROR:root

...

ERROR:root:Use meaningful ids if label not present: http://www.orpha.net/ORDO/Orphanet_100032 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: NCIT:C4380 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/SCTID_254955001 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://www.orpha.net/ORDO/Orphanet_99870 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: NCIT:C123317 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/OMIMPS_609192 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/UMLS_C0334607 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://www.orpha.net/ORDO/Orphanet_91136 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: DOID:12803 "None" label None No

ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/UMLS_C3151186 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/UMLS_C3642347 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/SCTID_719946008 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: DOID:0110110 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: NCIT:C67237 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/UMLS_C1335510 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/GARD_0012635 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: DOID:4230 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/UMLS_C0032460 "None" lab

ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/UMLS_C0042994 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: NCIT:C128047 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://www.orpha.net/ORDO/Orphanet_2863 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: DOID:5224 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/EFO_1000974 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: DOID:8149 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/SCTID_392300000 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/UMLS_C0206735 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: DOID:3710 "None" label None None 1.0
ERROR:root:Use meaningful 

ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/OMIM_606545 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/SCTID_707435002 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/SCTID_205480005 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: DOID:13487 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/COHD_252235 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: NCIT:C4277 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: DOID:0080101 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/MESH_D007172 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/OMIM_614800 "None" label N

ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/MESH_C567712 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/MESH_C567827 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://www.orpha.net/ORDO/Orphanet_519 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://www.orpha.net/ORDO/Orphanet_180253 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/SCTID_94140004 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://www.orpha.net/ORDO/Orphanet_91144 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/MESH_D011662 "None" label None None 1.0
ERROR:root:Use meaningful ids if label not present: http://purl.obolibrary.org/obo/OMIM_223900 "None" label None None 1.0
ERROR:root:Use meaningful

In [14]:
## CONFIGURE
## we will map R to mondo and hp separately
lexmap.ontology_pairs = [(rare.id, mondo.id), (rare.id, hp.id)]

In [15]:
# align
g = lexmap.get_xref_graph()

In [17]:
# get a dataframe from the mapping graph
df=lexmap.as_dataframe(g)
df

,left,left_label,right,right_label,left_match_type,right_match_type,left_match_val,right_match_val,score,left_simscore,...,conditional_pr_equiv,pr_subClassOf,pr_superClassOf,pr_equivalentTo,pr_other,left_novel,right_novel,left_consistent,right_consistent,equiv_clique_size
3287,11-beta-hydroxylase deficiency,11-beta-hydroxylase deficiency,MONDO:0008729,congenital adrenal hyperplasia due to 11-beta-...,label,hasRelatedSynonym,11-beta-hydroxylase deficiency,11-Beta-Hydroxylase Deficiency,50.0,1.000000,...,1.000000,0.061581,0.061581,0.799654,0.077184,True,True,False,False,7
2199,15q13.3 microdeletion syndrome,15q13.3 microdeletion syndrome,MONDO:0012774,chromosome 15q13.3 microdeletion syndrome,label,hasExactSynonym,15q13.3 microdeletion syndrome,15q13.3 microdeletion syndrome,90.0,1.000000,...,1.000000,0.029969,0.029969,0.918763,0.021299,True,True,False,False,6
3339,17-alpha-hydroxylase deficiency,17-alpha-hydroxylase deficiency,MONDO:0008730,congenital adrenal hyperplasia due to 17-alpha...,label,hasRelatedSynonym,17-alpha-hydroxylase deficiency,17-Alpha-Hydroxylase Deficiency,50.0,1.000000,...,1.000000,0.061581,0.061581,0.799654,0.077184,True,True,False,False,5
3481,17-beta hydroxysteroid dehydrogenase 3 deficiency,17-beta hydroxysteroid dehydrogenase 3 deficiency,MONDO:0009916,"46,XY disorder of sex development due to 17-be...",label,hasExactSynonym,17-beta hydroxysteroid dehydrogenase 3 deficiency,17-beta-hydroxysteroid dehydrogenase 3 deficiency,58.0,1.000000,...,1.000000,0.205965,0.205965,0.392394,0.195675,True,True,False,False,7
2592,17q21.31 microdeletion syndrome,17q21.31 microdeletion syndrome,MONDO:0012496,Koolen de Vries syndrome,label,hasExactSynonym,17q21.31 microdeletion syndrome,17q21.31 microdeletion syndrome,90.0,1.000000,...,0.473684,0.168017,0.055554,0.749591,0.026839,True,True,False,False,8
2593,17q21.31 microdeletion syndrome,17q21.31 microdeletion syndrome,MONDO:0018216,17q21.31 microdeletion syndrome,label,label,17q21.31 microdeletion syndrome,17q21.31 microdeletion syndrome,100.0,1.000000,...,0.526316,0.051671,0.108232,0.824734,0.015363,True,True,False,False,8
2987,18 Hydroxylase deficiency,18 Hydroxylase deficiency,MONDO:0008751,Corticosterone methyloxidase type 1 deficiency,label,hasRelatedSynonym,18 Hydroxylase deficiency,18-Hydroxylase Deficiency,32.0,1.000000,...,0.355556,0.232996,0.289482,0.283582,0.193941,True,True,False,False,6
2986,18 Hydroxylase deficiency,18 Hydroxylase deficiency,MONDO:0020489,familial hyperreninemic hypoaldosteronism type 1,label,hasExactSynonym,18 Hydroxylase deficiency,18-hydroxylase deficiency,58.0,1.000000,...,0.644444,0.292046,0.210145,0.309167,0.188643,True,True,False,False,6
1960,1q21.1 microdeletion syndrome,1q21.1 microdeletion syndrome,MONDO:0012914,chromosome 1q21.1 deletion syndrome,label,hasExactSynonym,1q21.1 microdeletion syndrome,1q21.1 microdeletion syndrome,90.0,1.000000,...,1.000000,0.030109,0.030109,0.923042,0.016740,True,True,False,False,6
1428,2 4-Dienoyl-CoA reductase deficiency,2 4-Dienoyl-CoA reductase deficiency,MONDO:0014464,progressive encephalopathy with leukodystrophy...,label,hasExactSynonym,2 4-Dienoyl-CoA reductase deficiency,"2,4-dienoyl-CoA reductase deficiency",58.0,1.000000,...,1.000000,0.200803,0.200803,0.382559,0.215835,True,True,False,False,5


In [18]:
## write to file (not used here but can be examined separately)
df.to_csv('rare-matches.tsv', sep="\t", index=False)

In [27]:
udf = lexmap.unmapped_dataframe(g)

In [28]:
## unmapped (TODO this includes unmapped from MONDO/HP to R, which we don't care about so much)
udf.to_csv('rare-no-matches.tsv', sep="\t", index=False)
udf

,id,label,mapped_equivs
67812,16p11.2 deletion syndrome,16p11.2 deletion syndrome,
41310,2-Methylacetoacetyl CoA thiolase deficiency,2-Methylacetoacetyl CoA thiolase deficiency,
65957,2-hydroxyethyl methacrylate sensitization,2-hydroxyethyl methacrylate sensitization,
3885,22q11.2 duplication syndrome,22q11.2 duplication syndrome,
16868,22q13.3 deletion syndrome,22q13.3 deletion syndrome,
23584,2q37 deletion syndrome,2q37 deletion syndrome,
63036,3 Methylcrotonyl-CoA carboxylase 1 deficiency,3 Methylcrotonyl-CoA carboxylase 1 deficiency,
41254,3 alpha methylcrotonyl-CoA carboxylase 2 defic...,3 alpha methylcrotonyl-CoA carboxylase 2 defic...,
55462,3-alpha hydroxyacyl-CoA dehydrogenase deficiency,3-alpha hydroxyacyl-CoA dehydrogenase deficiency,
22117,3p deletion syndrome,3p deletion syndrome,
